# Dedalus script for 2D tracer adv-dif on a slope Analysis
## Ryan Holmes

In [ ]:
# Load modules
import numpy as np
import h5py
import matplotlib
import shutil
import os
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
matplotlib.rcParams["figure.facecolor"] = "white"
matplotlib.rcParams["axes.facecolor"] = "white"
matplotlib.rcParams["savefig.facecolor"] = "white"
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
from mpi4py import MPI
from scipy.special import erf
from scipy.interpolate import spline
from scipy import signal
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

from dedalus.tools import post
import pathlib
from dedalus.extras import plot_tools

import obsfit1d
import bbin

## Analyse runs

In [ ]:
# basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-5-19/';
# savedir = '/home/z3500785/Cloud/Research/Data_Analysis/Bottom_Mixing_McDougall/Slope_Tracer_dedalus/Analysis_Figures/prodruns24-5-19/'
basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns_wide30-5-19/';
savedir = '/home/z3500785/Cloud/Research/Data_Analysis/Bottom_Mixing_McDougall/Slope_Tracer_dedalus/Analysis_Figures/prodruns_wide30-5-19/'
# basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns_bIC30-5-19/';
# savedir = '/home/z3500785/Cloud/Research/Data_Analysis/Bottom_Mixing_McDougall/Slope_Tracer_dedalus/Analysis_Figures/prodruns_bIC30-5-19/'


In [ ]:
# display file contents
with h5py.File(basedir + 'z0_2_AH_000_slope_100/snapshots.h5', mode='r') as file:
    print(list(file['scales'].keys()))
    print(file['tasks/tr'])

### Plot Input fields

In [ ]:
#runs = [basedir + 'z0_0p5000_AH_000/'];
runs = [basedir + 'z0_0p5000_AH_000_Kinf_m%s_ADV_2_slope_400/' % x for x in [3,4,5]] + [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_100/']# + [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [1]]
#labs = ['Constant $\kappa=10^{-3}$m$^2$s$^{-1}$','$\kappa_\infty=10^{-4}$m$^2$s$^{-1}$','$\kappa_\infty=10^{-5}$m$^2$s$^{-1}$','$\kappa_\infty=10^{-5}$m$^2$s$^{-1}$, slope = 1/100','$\kappa_\infty=10^{-5}$m$^2$s$^{-1}$ no $V^{SML}$']
#runs = [basedir + ('z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_%03d/' % x) for x in [100,200,400]]
#labs = ['1/100','1/200','1/300']
params = np.load(runs[0] + 'runparams.npz');locals().update(params)
colors = ['k.-','r.-','b.-','g.-','m.','y.-'];

# Setup figure:
fig = plt.figure(figsize=(11, 5), facecolor='w')
axisV = plt.subplot(1,3,1)
axisV.set_xlabel(r'$V$ (cms$^{-1}$)')
axisV.set_ylabel(r'z (m)')
axisV.set_ylim([0.,Lz/5.])
axisV.grid()
axisV.set_title('Upslope Velocity')
axisB = plt.subplot(1,3,2)
axisB.set_xlabel(r'$\frac{\partial B}{\partial z}$ ($10^{-6}$s$^{-2}$)')
axisB.set_ylim([0.,Lz/5.])
axisB.axes.get_yaxis().set_ticklabels([])
axisB.grid()
axisB.set_title('Buoyancy Frequency')
axisK = plt.subplot(1,3,3)
axisK.set_xlabel(r'$\kappa$ (cm$^2$s$^{-1}$)')
axisK.set_ylim([0.,Lz/5.])
axisK.axes.get_yaxis().set_ticklabels([])
axisK.grid()
axisK.set_title('Isotropic Diffusivity')
panlabs = ['a','b','c','d','e','f','g','h','i','j','k','l']


for ii in range(0,len(runs)):
    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    with h5py.File(runs[ii] + 'ifields.h5', mode='r') as file:
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
        V = file['tasks/V'][0,0,:];K = file['tasks/K'][0,0,:];
        Bz = file['tasks/Bz'][0,0,:];
        axisV.plot(V*100.0,z.T,colors[ii], linewidth=2.,label=labs[ii]);
        axisB.plot(Bz*1.e6,z.T,colors[ii], linewidth=2.);
        axisK.plot((K)/1.0e-4,z.T,colors[ii], linewidth=2.);
#axisV.legend(title='Slope')

xlims = axisV.get_xlim();ylims = axisV.get_ylim();
axisV.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.93*(ylims[1]-ylims[0]),panlabs[0],fontsize=25,fontweight='bold')
xlims = axisB.get_xlim();ylims = axisB.get_ylim();
axisB.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.93*(ylims[1]-ylims[0]),panlabs[1],fontsize=25,fontweight='bold')
xlims = axisK.get_xlim();ylims = axisK.get_ylim();
axisK.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.93*(ylims[1]-ylims[0]),panlabs[2],fontsize=25,fontweight='bold')

axisV.legend()
plt.tight_layout()
plt.savefig(savedir + 'Input_VKBz.pdf')


### Plot 2D Tracer Distribution

In [ ]:
# Simple plotting:
#names = ['z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400' for x in ['2p0000','1p0000','0p5000','0p2500','0p1250','0p0625']]
#names = ['z0_0p5000_AH_000_Kinf_m%s_ADV_2_slope_400' % x for x in [3,4,5]] + ['z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400' % x for x in [0,1]]
#names = ['z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_%03d' % x for x in [100,200,400]]
#names = ['z0_0p5000_AH_%03d_Kinf_m5_ADV_2_slope_400' % x for x in [0,1,5,10,50,100]]
#names = ['z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400']
names = ['z0_2_AH_000_slope_100'];
titles = ['No Advection']

for fi in range(0,len(names)):
    name = names[fi]
    outdir = basedir + name + '/'
    params = np.load(outdir + 'runparams.npz');locals().update(params)
    snaps = outdir + 'snapshots.h5';
    fig = plt.figure(figsize=(20, 8), facecolor='w')
    tts = [round(x) for x in np.linspace(0,Itot/sfreq-1,4)]
    for ii in range(len(tts)):#range(0,49):
        axisTR = plt.subplot(2,2,ii+1)
        axisTR.set_xlabel(r'True y (km)');axisTR.set_ylabel(r'True z (m)');axisTR.set_title(r'Tracer Concentration')
        axisTR.set_xlim([0,Ly/1.e3]);axisTR.set_ylim([0.,Lz + slope*Ly]);axisTR.set_facecolor('k')

        with h5py.File(snaps, mode='r') as file:
            y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
            dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
            dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);
            ym, zm = np.meshgrid(y,z)
            zt = np.cos(theta)*zm + np.sin(theta)*ym;
            yt = -np.sin(theta)*zm + np.cos(theta)*ym # Rotated Coordinates
            B = N2*np.sin(theta)*ym + N2*np.cos(theta)*(zm + np.exp(-q0*zm)*np.cos(q0*zm)/q0)
            t = file['scales/sim_time'][:]/lday;
            trT = file['tasks']['trT'][:,0,0]
            tr = file['tasks']['tr'][tts[ii],:,:]
#            p = axisTR.pcolormesh(yt/1.e3, zt, tr.T, cmap='RdBu', vmin=0., vmax=0.15)#np.max(tr)/);#GnBu
            p = axisTR.contourf(yt/1.e3, zt, tr.T, cmap='magma_r', levels=np.arange(0.,np.max(tr),np.max(tr)/26),extend='both')#np.max(tr)/);#GnBu

        # Add buoyancy contours
        plt.contour(yt/1.0e3, zt, B, 15, colors='k')
        axisTR.plot(y/1.0e3, slope*y,'k-', linewidth=4.)
        plt.colorbar(p, ax = axisTR)
        plt.title(titles[fi] + ('Day %d' % t[tts[ii]]))
        ax = plt.gca()
#        ax.set_ylim([3000.,6000.])
        # ax.set_xlim([100.,900.])

    plt.tight_layout()
#    plt.savefig(savedir + name + 'TrConcMgma.png')    


In [ ]:
#ls /srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns_wide30-5-19/z0_0p5000_AH100_ADV_0_Kinf_m3/
!ls /srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns_wide30-5-19

In [ ]:
# Combination Plotting:
# names = ['z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400' for x in ['1p0000','0p5000','0p1250']]
# titles = ['$z_0=d=500$m','$z_0=d/2=250$m','$z_0=d/8=62.5$m']
#names = ['z0_0p5000_AH_000_Kinf_m%s_ADV_2_slope_400' % x for x in [3,4,5]] + ['z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400' % x for x in [0,1]]
# names = ['z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400' % x for x in [0,1,2]]
# titles = ['No Advection','BBL Advection Only','SML and BBL Advection']
# names = ['z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_%03d' % x for x in [100,200,400]]
# titles = ['Slope = 1/100','Slope = 1/200','Slope = 1/400']
# names = ['z0_0p5000_AH_%03d' % x for x in [0,10,100]]
# titles = ['No Isopycnal Diffusion','$A_H=10$m$^2$s$^{-1}$','$A_H=100$m$^2$s$^{-1}$']
# names = ['z0_0p5000_AH_100_ADV_%01d' % x for x in [0,1]]
# names = names + ['z0_0p5000_AH_100']
# titles = ['No Advection','BBL Advection Only','SML and BBL Advection']
names = ['z0_0p5000_AH_100_ADV_0_Kinf_m3','z0_0p5000_AH_100_ADV_0','z0_0p5000_AH_010_ADV_0']
titles = [r'Constant $\kappa$, $A_H=100$m$^2$s$^{-1}$',r'Bottom-intensified $\kappa$, $A_H=100$m$^2$s$^{-1}$',r'Bottom-intensified $\kappa$, $A_H=10$m$^2$s$^{-1}$']
# titles = ['No Isopycnal Diffusion','$A_H=10$m$^2$s$^{-1}$','$A_H=100$m$^2$s$^{-1}$']
# names = ['z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400']
# names = ['z0_' + x + '_AH_100' for x in ['2p0000','1p0000','0p5000']]
# titles = ['$z_0=2d=1000$m','$z_0=d=500$m','$z_0=d/2=250$m']
# names = ['ADV_' + x + '_AH_100' for x in ['0','1','2']]
# titles = ['No Advection','BBL Advection Only','SML and BBL Advection']
# names = ['ADV_2_AH_000_slope_%03d' % x for x in [100, 200]]
# titles = ['Slope = 1/100','Slope = 1/200']
# names = ['ADV_0_AH_000_slope_200_maxy_%03d' % x for x in [800,850]]
# titles = ['maxy = 800km','maxy = 850km']
# names = ['ADV_%01d_AH_100_slope_200_maxy_10000000_miny_600' % x for x in [0,1,2]]
# titles = ['No Advection $A_H=100$m$^2$s$^{-1}$','BBL Advection Only $A_H=100$m$^2$s$^{-1}$','SML and BBL Advection $A_H=100$m$^2$s$^{-1}$']

panlabs = ['a','b','c','d','e','f','g','h','i','j','k','l']

plts = [1,4,2,5,3,6]
#plts = [1,4,7,2,5,8,3,6,9]
fig = plt.figure(figsize=(20, 8), facecolor='w')
for fi in range(0,len(names)):
    name = names[fi]
    outdir = basedir + name + '/'
    params = np.load(outdir + 'runparams.npz');locals().update(params)
    snaps = outdir + 'snapshots.h5';
#    tts = [round(x) for x in np.linspace(0,Itot/sfreq*3./4.-1,4)]
    # tts = [100, 200-1]
    tts = [50, 100]
#    tts = [10, 20]
    #tts = [round(x) for x in [Itot/sfreq-1]]
    for ii in range(len(tts)):#range(0,49):
        axisTR = plt.subplot(2,3,plts[fi*2+ii])
        axisTR.set_xlabel(r'True y (km)');axisTR.set_ylabel(r'True z (m)');axisTR.set_title(r'Tracer Concentration')
        axisTR.set_xlim([0,Ly/1.e3]);axisTR.set_ylim([0.,Lz + slope*Ly]);axisTR.set_facecolor('k')
        with h5py.File(snaps, mode='r') as file:
            y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
            dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
            dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);
            ym, zm = np.meshgrid(y,z)
            zt = np.cos(theta)*zm + np.sin(theta)*ym;
            yt = -np.sin(theta)*zm + np.cos(theta)*ym # Rotated Coordinates
            B = N2*np.sin(theta)*ym + N2*np.cos(theta)*(zm + np.exp(-q0*zm)*np.cos(q0*zm)/q0)
            t = file['scales/sim_time'][:]/lday;
            trT = file['tasks']['trT'][:,0,0]
            tr = file['tasks']['tr'][tts[ii],:,:]
            tr0 = file['tasks']['tr'][0,:,:]
            zm1 = file['tasks']['zm1i'][tts[ii],0,0];
            muz = zm1/trT
            ym1 = file['tasks']['ym1i'][tts[ii],0,0];
            muy = ym1/trT
            if fi == 2:
                mx = 0.05
            else:
                mx = 0.02
            p = axisTR.contourf(yt/1.e3, zt, tr.T, cmap='magma_r', levels=np.arange(0.,mx+mx/40,mx/40),extend='both')#np.max(tr)/);#GnBu
            # PO = axisTR.contour(yt/1.e3, zt, tr0.T, colors='g', levels=[0.5], linewidths=2)#np.max(tr)/);#GnBu
            # cbar = plt.colorbar(PCM, ax=axisTR)
            # cbar.set_ticks(np.arange(0.,mx,mx/5))
            # cbar.set_ticklabels(np.arange(0.,mx,mx/5))
            plt.plot((-np.sin(theta)*cz + np.cos(theta)*cy)/1e3,np.cos(theta)*cz+np.sin(theta)*cy,'og',mew=4,ms=12)
            plt.plot((-np.sin(theta)*muz + np.cos(theta)*muy)/1e3,np.cos(theta)*muz+np.sin(theta)*muy,'xb',mew=4,ms=12)
            
        # Add buoyancy contours
        plt.contour(yt/1.0e3, zt, B, 30, colors='k')
        axisTR.plot(y/1.0e3, slope*y,'k-', linewidth=4.)
        plt.colorbar(p, ax = axisTR)
        plt.title(titles[fi] + ('  Day %d' % t[tts[ii]]))
        axisTR.set_ylim([1000.,3500.])
        axisTR.set_xlim([200.,1200.])
        # if fi == 0:
        #     axisTR.set_ylim([3000.,5000.])
        # elif fi == 1:
        #     axisTR.set_ylim([1500.,3500.])
        # else:
        #     axisTR.set_ylim([500., 2500.])
        # axisTR.set_xlim([150.,700.])
        # axisTR.set_ylim([3200.,6000.])
        # axisTR.set_xlim([300.,1100.])
        xlims = axisTR.get_xlim()
        ylims = axisTR.get_ylim()
        txt = axisTR.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.915*(ylims[1]-ylims[0]),panlabs[plts[fi*2+ii]-1],fontsize=25,fontweight='bold')
#        txt.set_color('white')

plt.tight_layout()
plt.savefig(savedir + 'TrConcMgma_NOADV_KinfAHCOMP.png',dpi=200)    


### Buoyancy distribution and moments

In [ ]:
np.sqrt(2*np.pi)-1

In [ ]:
fig = plt.figure(figsize=(10, 7.5), facecolor='w')
ab = [];
nex = 3;
ab.append(plt.subplot2grid((nex,2),(0,0), colspan=1, rowspan=nex))
for i in np.arange(1,nex+1):
    ab.append(plt.subplot2grid((nex,2),(i-1,1)));
ab[0].set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');ab[0].set_xlabel('Tracer Concentration (tr / (ms$^{-2}$))');
ab[0].set_title('Tracer Distribution')
ab[1].set_ylabel(r'$\mu_b\,\, (10^{-4} ms^{-2})$');#ab[1].text(set_title('Centre of mass $\mu_b$');
ab[2].set_ylabel(r'$\kappa\,\, (cm^2s^{-1})$');#ab[3].set_title('Effective diffusivity from second moment');
ab[3].set_ylabel(r'$\overline{\kappa}^t\,\, (cm^2s^{-1})$');#ab[3].set_title('Effective diffusivity from second moment');
ab[nex].set_xlabel('Days')
for i in np.arange(1,nex+1):
    ab[i].grid();
panlabs = [r'a',r'b',r'c',r'd',r'e',r'f',r'g',r'h',r'i',r'j',r'k',r'l']

# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
# runs = [basedir + 'z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400/' for x in ['1p0000','0p5000','0p1250']]
# leglabel = ['$z_0=d=500$m','$z_0=d/2=250$m','$z_0=d/8=62.5$m']
# runs = [basedir + 'z0_0p5000_AH_%03d/' % x for x in [0,10,100]]
# leglabel = ['No Isopycnal Diffusion','$A_H=10$m$^2$s$^{-1}$','$A_H=100$m$^2$s$^{-1}$']
# runs = [basedir + 'ADV_%s_AH_000_slope_200/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
# runs = [basedir + 'ADV_%01d_AH_000_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]]
# runs = runs + [basedir + 'ADV_%01d_AH_100_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection $A_H=0$m$^2$s$^{-1}$','BBL Advection Only $A_H=0$m$^2$s$^{-1}$','No Advection $A_H=0$m$^2$s$^{-1}$']
# leglabel = leglabel + ['SML and BBL Advection $A_H=100$m$^2$s$^{-1}$','BBL Advection Only $A_H=100$m$^2$s$^{-1}$','No Advection $A_H=100$m$^2$s$^{-1}$']
# runs = [basedir + 'z0_0p5000_AH_%03d_ADV_0_Kinf_m3/' % x for x in [100]]
# runs = runs + [basedir + 'z0_0p5000_AH_100_ADV_%01d/' % x for x in [0,1]];
# runs = runs + [basedir + 'z0_0p5000_AH_100/']
# leglabel = ['Constant Isotropic Diffusivity','Bottom-Intensified Isotropic Diffusivity']#,'Kinf = -5']
# leglabel = leglabel + ['Bot-Intens. + BBL Advection','Bot-Intens. + SML + BBL Advection']
runs = [basedir + 'z0_0p5000_AH_100/'] + [basedir + 'z0_0p5000_AH_100_ADV_%01d/' % x for x in [1,0]]
leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']

# runs = [basedir + 'z0_0p5000_AH_100_ADV_0_Kinf_m3/',basedir + 'z0_0p5000_AH_100_ADV_0/',basedir + 'z0_0p5000_AH_010_ADV_0/']
# leglabel = [r'Constant $\kappa$, $A_H=100$m$^2$s$^{-1}$',r'Bottom-intensified $\kappa$, $A_H=100$m$^2$s$^{-1}$',r'Bottom-intensified $\kappa$, $A_H=10$m$^2$s$^{-1}$']

# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_%03d/' % x for x in [100,200,400]]
# leglabel = ['1/100','1/200','1/400']
# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400' + x + '/' for x in ['','_4dt','_4dt_2dz']]# + [basedir + 'z0_0p5000_AH_050_Kinf_m5_ADV_2_slope_400' + x + '/' for x in ['','_4dt']]
# leglabel = ['$\Delta t=8$ days, 128 z modes','$\Delta t=2$ days, 128 z modes','$\Delta t=2$ days, 256 z modes']#'$\Delta t=8$ days, $A_H=50$m$^2$s$^{-1}$','$\Delta t=2$ days, $A_H=50$m$^2$s$^{-1}$']
# runs = [basedir + 'ADV_2_AH_100_slope_%03d/' % x for x in [100, 200]]
# leglabel = ['Slope = 1/100','Slope = 1/200']
colors = ['g-','r-','b-','k-','c-','g--','r--','b--'];

# tifin = 200 # Final time index
# tinds = [x//16 for x in [0,1000,2000,3000]]
tifin = 101 # Final time index
tinds = [x//16 for x in [0,200,400,1600]]

dop3f = True#False
doKhw = False

if doKhw:
    fig2, ax = plt.subplots()
    fig2.set_facecolor('white')

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    
    if dop3f:
        par3fits = np.load(runs[ii] + 'par3fitGOODINI.npz');
        sucinds = par3fits['suc'] == 1

    with h5py.File(runs[ii] + 'ifields.h5', mode='r') as file:
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        trT = file['tasks']['trT'][:,0,0]
        trI = file['tasks']['tr'][0,:,:]
        b, trIp = bbin.trbinI(y,z,trI,N2,theta,Lz,Ly,q0)
        zF = b/N2;dzF = zF[1]-zF[0];
        imax = np.argmax(trIp)
        zF = zF - zF[imax]

        xlims = [0.,t[tifin-1]]
        for i in tinds:
            tr = file['tasks']['tr'][i,:,:]
            b, trp = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0)
            if i == 0:
                ab[0].plot(signal.savgol_filter(trp,5,2),b/1.e-4,colors[ii],label=leglabel[ii],linewidth=2)
            else:
                ab[0].plot(signal.savgol_filter(trp,5,2),b/1.e-4,colors[ii],linewidth=2)
                if (dop3f):
                    trfit = obsfit1d.trMOD([par3fits['K'][i],par3fits['Kh'][i],
                                            par3fits['w'][i]],zF,trp,trIp, int(t[i]*lday))
                    # ab[0].plot(trfit,b/1.e-4,colors[ii].replace('-',':'),linewidth=2)

        zm1 = file['tasks']['zm1i'][:,0,0];
        muz = zm1/trT

        bm1 = file['tasks']['bm1i'][:,0,0];bm2 = file['tasks']['bm2i'][:,0,0]
        Ktr = file['tasks']['Ktr'][:,0,0];Katrz = file['tasks']['Katrz'][:,0,0];atrzT = file['tasks']['atrzT'][:,0,0]
        mu = bm1/trT
        sigma2 = bm2/trT-(bm1/trT)**2.
        sigma2nomu = bm2/trT
        Keff = 0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/N2/N2/1.e-4;
        Ktrw = Ktr/trT/1e-4
        Ktrgw = Katrz/atrzT/1e-4
        ab[1].plot(t,mu/1.e-4,colors[ii],linewidth=2)
        ab[2].plot((t[1:]+t[:-1])/2,Keff,colors[ii],linewidth=2)
        ab[2].plot(t,Ktrw,colors[ii].replace('-','--'),linewidth=2)
#        ab[2].plot(t,Ktrgw,colors[ii].replace('-',':'),linewidth=3)
        KeffTA = [Keff[0]] + [np.sum(np.array(Keff[:i+1])/(i+1)) for i in range(0,len(Keff))]
        KtrwTA = [np.sum(np.array(Ktrw[:i+1])/(i+1)) for i in range(0,len(Ktrw))]
        KtrgwTA = [np.sum(np.array(Ktrgw[:i+1])/(i+1)) for i in range(0,len(Ktrgw))]
        ab[3].plot(t,KeffTA,colors[ii],linewidth=2);
        ab[3].plot(t,KtrwTA,colors[ii].replace('-','--'),linewidth=2);
        if (dop3f):
            p3t = par3fits['t']
            p3K = np.array([x/1e-4 for x in par3fits['K']])
            ab[3].plot(p3t[sucinds],p3K[sucinds],colors[ii].replace('-','-.').replace(':','-.'),linewidth=2);
            #ab[3].plot(p3t[1:],p3K[1:],colors[ii].replace('-','-.'),linewidth=2);
            # Kp = (np.array(par3fits['K'])+2*np.array(par3fits['Kerr']))/1e-4
            # Km = (np.array(par3fits['K'])-2*np.array(par3fits['Kerr']))/1e-4
            # ab[3].plot(par3fits['t'],Kp,colors[ii].replace('-','-.'));
            # ab[3].plot(par3fits['t'],Km,colors[ii].replace('-','-.'));
            if doKhw:
                p3Kh = np.array([-x/1e-7 for x in par3fits['Kh']]) # I had the sign of D1 wrong!!
                p3w = np.array([-x/1e-7 for x in par3fits['w']]) # 
                if ( i==0 ):
                    ax.plot(p3t[1:],p3Kh[1:],colors[ii],linewidth=2,label='\overline{\kappa}_h');
                    ax.plot(p3t[1:],p3w[1:],colors[ii].replace('-','--'),linewidth=2,label='\overline{w}');
                else:
                    ax.plot(p3t[1:],p3Kh[1:],colors[ii],linewidth=2);
                    ax.plot(p3t[1:],p3w[1:],colors[ii].replace('-','--'),linewidth=2);

ab[0].legend()
# # ADV case:
# ab[0].set_ylim([5.,25.]);ab[0].set_xlim([0.,1.e10])
# ab[1].set_ylim([10.,18.]);ab[2].set_ylim([0.,7.5]);
# ab[3].set_ylim([0.,7.5])
# # z0 case:
# ab[0].set_ylim([6.,18.]);ab[0].set_xlim([0.,1.8e10])
# ab[1].set_ylim([8.,15.]);ab[2].set_ylim([0.,10.]);
# ab[3].set_ylim([0.,10.])
# AH100 ADV case:
ab[0].set_ylim([15.,28.]);ab[0].set_xlim([0.,1.3e10])
ab[1].set_ylim([20.,23.]);ab[2].set_ylim([0.,7.]);
ab[3].set_ylim([0.,7.])
# # AH100 NOADV case:
# ab[0].set_ylim([15.,28.]);ab[0].set_xlim([0.,1.3e10])
# ab[1].set_ylim([20.,22.5]);ab[2].set_ylim([0.,11.]);
# ab[3].set_ylim([0.,11.])
# # Layer case:
# ab[0].set_ylim([38.,52.]);ab[0].set_xlim([0.,2.e11])
# ab[1].set_ylim([44.8,47.5]);ab[2].set_ylim([0.,6.]);
# ab[3].set_ylim([0.,5.])
# # Layer case AH=100:
# ab[0].set_ylim([32.,56.]);ab[0].set_xlim([0.,0.6e11])
# ab[1].set_ylim([42.5,46.5]);ab[2].set_ylim([0.,10.]);
# # AH case:
# ab[0].set_ylim([15.,28.]);ab[0].set_xlim([0.,1.3e10])
# ab[1].set_ylim([21.,23.5]);ab[2].set_ylim([0.,8.]);
# ab[3].set_ylim([0.,8.])

for i in np.arange(1,nex+1):
    ab[i].set_xlim(xlims);
    xlims = ab[i].get_xlim();ylims = ab[i].get_ylim();
    ab[i].text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.88*(ylims[1]-ylims[0]),panlabs[i],fontsize=15,fontweight='bold')
for i in np.arange(1,nex):
    ab[i].axes.xaxis.set_ticklabels([])

xlims = ab[0].get_xlim();ylims = ab[0].get_ylim();
ab[0].text(xlims[0]+0.08*(xlims[1]-xlims[0]),ylims[0]+0.96*(ylims[1]-ylims[0]),panlabs[0],fontsize=15,fontweight='bold')

if doKhw:
    ax.set_xlabel('Days')
    ax.set_ylabel(r'$(cm day^{-1})$');#ab[3].set_title('Effective diffusivity from second moment');
    ax.grid()
    ax.set_xlim([0., 2000.])
    ax.set_title(r'1D fit $\overline{w}$ (dashed) and $\overline{\kappa}_h$ (solid)')

# ab[0].set_title('Tracer Distribution Days 0 and 1600')

# ab[4].set_xlim([0., 1000.])
# ab[4].set_ylim([-50., 25.])
# plt.tight_layout()
plt.savefig(savedir + 'bSpaceAnalysis_AH100_ADV.pdf');


In [ ]:
# B distribution and moments Large time series
fig = plt.figure(figsize=(8, 12), facecolor='w')
ab = []
for i in range(0,5):
    ab.append(plt.subplot2grid((5,1),(i,0)))
ab[0].set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');
ab[1].set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');
ab[2].set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');
ab[0].grid();ab[1].grid();ab[2].grid();ab[3].grid();ab[4].grid()
ab[3].set_ylabel(r'Time-Average $\kappa (cm^2s^{-1})$');
ab[4].set_ylabel(r'Time-Average $\overline{w}$, $\overline{\kappa}_h$ (cm day$^{-1}$)');
ab[4].set_xlabel('Days')

runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [2,1,0]]
runs = [runs[1]]
leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
colors = ['g-','r-','b-','k-','y-','m-'];
panlabs = ['a ' + leglabel[0],'b ' + leglabel[1],'c ' + leglabel[2],'d','e','f','g','h','i','j','k','l']

# pfr = 10 # Profile frequency
# tifin = 100 # Final time index
# concsc = 200./1.e10 # Concentration to time conversion scale
pfr = 20 # Profile frequency
tifin = 199 # Final time index
concsc = 400./1.e10 # Concentration to time conversion scale

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    par3fits = np.load(runs[ii] + 'par3fitGOODINI.npz');
    sucinds = par3fits['suc'] == 1

    with h5py.File(runs[ii] + 'ifields.h5', mode='r') as file:
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        trT = file['tasks']['trT'][:,0,0]
        trI = file['tasks']['tr'][0,:,:]
        b, trIp = bbin.trbinI(y,z,trI,N2,theta,Lz,Ly,q0)
        zF = b/N2;dzF = zF[1]-zF[0];
        imax = np.argmax(trIp)
        zF = zF - zF[imax]

        xlims = [0.,t[tifin]]
        for i in range(0,tifin,pfr):#len(t),pfr):
            tr = file['tasks']['tr'][i,:,:]
            b, trp = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0)
            ab[ii].plot(signal.savgol_filter(trp,5,2)*concsc+t[i],b/1.e-4,colors[ii])
            if (i > 1):
                trfit = obsfit1d.trMOD([par3fits['K'][i],par3fits['Kh'][i],
                                        par3fits['w'][i]],zF,trp,trIp, int(t[i]*lday))
                ab[ii].plot(signal.savgol_filter(trfit,5,2)*concsc+t[i],b/1.e-4,colors[ii].replace('-','--'))
            
        zm1 = file['tasks']['zm1i'][:,0,0];
        muz = zm1/trT

        bm1 = file['tasks']['bm1i'][:,0,0];bm2 = file['tasks']['bm2i'][:,0,0]
        Ktr = file['tasks']['Ktr'][:,0,0];Katrz = file['tasks']['Katrz'][:,0,0];atrzT = file['tasks']['atrzT'][:,0,0]
        mu = bm1/trT
        sigma2 = bm2/trT-(bm1/trT)**2.
        sigma2nomu = bm2/trT
        Keff = 0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/N2/N2/1.e-4;
        Ktrw = Ktr/trT/1e-4
        Ktrgw = Katrz/atrzT/1e-4
        ab[ii].plot(t,mu/1.e-4,colors[ii].replace('-',':'),linewidth=3)
        # ab[3].plot(t,np.sqrt(sigma2)/1.e-4,colors[ii])
        # ab[3].plot((t[1:]+t[:-1])/2,Keff,colors[ii])
        # ab[3].plot(t,Ktrw,colors[ii].replace('-','--'))
#        ab[4].plot(t,Katrgw,colors[ii].replace('-',':'),linewidth=3)
        KeffTA = [Keff[0]] + [np.sum(np.array(Keff[:i+1])/(i+1)) for i in range(0,len(Keff))]
        KtrwTA = [np.sum(np.array(Ktrw[:i+1])/(i+1)) for i in range(0,len(Ktrw))]
        KtrgwTA = [np.sum(np.array(Ktrgw[:i+1])/(i+1)) for i in range(0,len(Ktrgw))]
#        ab[3].plot(t,KeffTA,colors[ii],linewidth=2);
#        ab[3].plot(t,KtrwTA,colors[ii].replace('-','--'));
        p3t = par3fits['t']
        p3K = np.array([x/1e-4 for x in par3fits['K']])
        p3Kp = (np.array(par3fits['K'])+2*np.array(par3fits['Kerr']))/1e-4
        p3Km = (np.array(par3fits['K'])-2*np.array(par3fits['Kerr']))/1e-4
        ab[3].plot(p3t[sucinds],p3K[sucinds],colors[ii],linewidth=3);
        ab[3].plot(p3t[sucinds],p3Kp[sucinds],colors[ii],linewidth=1);
        ab[3].plot(p3t[sucinds],p3Km[sucinds],colors[ii],linewidth=1);
        p3Kh = np.array([-x/1e-7 for x in par3fits['Kh']]) # I had the sign of D1 wrong!!
        p3w = np.array([-x/1e-7 for x in par3fits['w']]) # 
        p3Khp = (-np.array(par3fits['Kh'])+2*np.array(par3fits['Kherr']))/1e-7
        p3Khm = (-np.array(par3fits['Kh'])-2*np.array(par3fits['Kherr']))/1e-7
        p3wp = (-np.array(par3fits['w'])+2*np.array(par3fits['werr']))/1e-7
        p3wm = (-np.array(par3fits['w'])-2*np.array(par3fits['werr']))/1e-7
        if (ii == 1):
            ab[4].plot(p3t[1:],p3Kh[1:],'-k',linewidth=3,label=r'$\overline{\kappa}_h$');
            ab[4].plot(p3t[1:],p3w[1:],'--k',linewidth=3,label=r'$\overline{w}$');
        ab[4].plot(p3t[1:],p3Kh[1:],colors[ii],linewidth=3);
        # ab[4].plot(p3t[1:],p3Khp[1:],colors[ii],linewidth=1);
        # ab[4].plot(p3t[1:],p3Khm[1:],colors[ii],linewidth=1);
        ab[4].plot(p3t[1:],p3w[1:],colors[ii].replace('-','--'),linewidth=3);
        # ab[4].plot(p3t[1:],p3w[1:],colors[ii].replace('-','--'),linewidth=1);
        # ab[4].plot(p3t[1:],p3w[1:],colors[ii].replace('-','--'),linewidth=1);

ab[4].legend()
for i in range(3):
    ab[i].set_ylim([7.5, 20.])
    
for i in range(5):
    ab[i].set_xlim(xlims)
    xlimsL = ab[i].get_xlim();ylimsL = ab[i].get_ylim();
    if i<=2:
        ab[i].text(xlimsL[0]+0.01*(xlimsL[1]-xlimsL[0]),ylimsL[0]+0.85*(ylimsL[1]-ylimsL[0]),panlabs[i],fontsize=15,fontweight='bold')
    else:
        ab[i].text(xlimsL[0]+0.01*(xlimsL[1]-xlimsL[0]),ylimsL[0]+0.05*(ylimsL[1]-ylimsL[0]),panlabs[i],fontsize=15,fontweight='bold')

for i in range(4):
    ab[i].axes.xaxis.set_ticklabels([])
# plt.tight_layout()
# plt.savefig(savedir + 'bSpaceAnalysis_ExampleFits.pdf');

# Summary Figure of effective diffusivity

In [ ]:
fig = plt.figure(figsize=(8, 5), facecolor='w')
ax = plt.gca()
ax.set_xlabel(r'Along-isopycnal Diffusivity (m$^2$s$^{-1}$)');
ax.grid()

base = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/'

runs = [base + 'prodruns24-5-19/z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [2,1,0]]
colors = ['g','r','b']
types = ['o','o','o']
labels = ['SML + BBL Adv','BBL Adv','No Adv']
runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_%03d/' % x for x in [5,10,50,100]])
colors.extend(['g'] * 4)
types.extend(['o'] * 4)
labels.extend([''] * 4)
runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_100_ADV_%01d/' % x for x in [1,0]])
colors.extend(['r','b'])
types.extend(['o','o'])
labels.extend(['']*2)
runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_005_ADV_%01d/' % x for x in [1,0]])
colors.extend(['r','b'])
types.extend(['o','o'])
labels.extend(['']*2)
runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_050_ADV_%01d/' % x for x in [1,0]])
colors.extend(['r','b'])
types.extend(['o','o'])
labels.extend(['']*2)
runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_010_ADV_%01d/' % x for x in [1,0]])
colors.extend(['r','b'])
types.extend(['o','o'])
labels.extend(['']*2)
runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_030_ADV_%01d/' % x for x in [2,1,0]])
colors.extend(['g','r','b'])
types.extend(['o','o','o'])
labels.extend(['']*3)
runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_075_ADV_%01d/' % x for x in [2,1,0]])
colors.extend(['g','r','b'])
types.extend(['o','o','o'])
labels.extend(['']*3)
# runs.extend([base + 'prodruns24-5-19/z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_%03d/' % x for x in [100,200]])
# colors.extend(['g','g'])
# types.extend(['1','2'])
# labels.extend(['Slope 1/100','Slope 1/200'])

# runs.extend([base + 'prodruns24-5-19/z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400/' for x in ['2p0000','1p0000','0p5000','0p1250','0p0625']])
# colors.extend(['g','g','g','g','g'])
# types.extend(['d','d','d','d','d'])

tind = 50
dop3f = True

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    
    if dop3f:
        par3fits = np.load(runs[ii] + 'par3fitGOODINI.npz');
        sucinds = par3fits['suc'] == 1

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        trT = file['tasks']['trT'][:,0,0]
        bm1 = file['tasks']['bm1i'][:,0,0];bm2 = file['tasks']['bm2i'][:,0,0]
        Ktr = file['tasks']['Ktr'][:,0,0];Katrz = file['tasks']['Katrz'][:,0,0];atrzT = file['tasks']['atrzT'][:,0,0]
        mu = bm1/trT
        sigma2 = bm2/trT-(bm1/trT)**2.
        sigma2nomu = bm2/trT
        Keff = 0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/N2/N2/1.e-4;
        Ktrw = Ktr/trT/1e-4
        Ktrgw = Katrz/atrzT/1e-4
        KeffTA = [Keff[0]] + [np.sum(np.array(Keff[:i+1])/(i+1)) for i in range(0,len(Keff))]
        KtrwTA = [np.sum(np.array(Ktrw[:i+1])/(i+1)) for i in range(0,len(Ktrw))]
        KtrgwTA = [np.sum(np.array(Ktrgw[:i+1])/(i+1)) for i in range(0,len(Ktrgw))]
        if (dop3f):
            p3t = par3fits['t']
            p3K = np.array([x/1e-4 for x in par3fits['K']])
        if (len(labels[ii])>0):
            ax.plot(AH,Keff[tind],types[ii],color=colors[ii],markersize=12,label=labels[ii])
        else:
            ax.plot(AH,Keff[tind],types[ii],color=colors[ii],markersize=12)
        if (ii == 0):
            ax.plot(AH,KtrwTA[tind],'x',color=colors[ii],markersize=12,linewidth=3,label='In-situ')
        else:
            ax.plot(AH,KtrwTA[tind],'x',color=colors[ii],markersize=12,linewidth=3)
        if (ii == 0):
            ax.plot(AH,par3fits['K'][tind]/1.e-4,'>',color=colors[ii],markersize=8,linewidth=3,label='Least-squares 1D')
        else:
            ax.plot(AH,par3fits['K'][tind]/1.e-4,'>',color=colors[ii],markersize=8,linewidth=3)
ax.legend()
ax.set_ylabel(r'$\overline{\kappa}^t$ Day %04d $(cm^2s^{-1})$' % t[tind]);

svdir = '/home/z3500785/Cloud/Research/Data_Analysis/Bottom_Mixing_McDougall/dedalus/Slope_Tracer/Analysis_Figures/'
plt.savefig(svdir + 'Summary_Plot_Day800.pdf');


### Z distribution and moments

In [ ]:
fig = plt.figure(figsize=(8, 6), facecolor='w')
# fig = plt.figure(figsize=(10, 5), facecolor='w')
ab0 = plt.subplot2grid((3,2),(0,0), colspan=1, rowspan=3)
ab1 = plt.subplot2grid((3,2),(0,1));
ab2 = plt.subplot2grid((3,2),(1,1));
ab3 = plt.subplot2grid((3,2),(2,1));
ab0.set_ylabel('z (m)');ab0.set_xlabel('Tracer Concentration (tr / m)');
ab0.set_title('Tracer Distribution');
ab1.set_ylabel(r'$\mu_z\,\, (m)$');#ab[1].text(set_title('Centre of mass $\mu_b$');
ab2.set_ylabel(r'$\sigma_z\,\, (m)$');#ab[1].text(set_title('Centre of mass $\mu_b$');
ab3.set_ylabel(r'$\kappa\,\, (cm^2s^{-1})$');#ab[3].set_title('Effective diffusivity from second moment');
ab1.grid();ab2.grid();ab3.set_xlabel('Days');ab3.grid()
panlabs = ['a','b','c','d','e','f','g','h','i','j','k','l']

# runs = [basedir + 'z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400/' for x in ['1p0000','0p5000','0p1250']]
# leglabel = ['$z_0=d=500$m','$z_0=d/2=250$m','$z_0=d/8=62.5$m']
# runs = [basedir + 'z0_0p5000_AH_%03d/' % x for x in [0,10,100]]
# leglabel = ['No Isopycnal Diffusion','$A_H=10$m$^2$s$^{-1}$','$A_H=100$m$^2$s$^{-1}$']
runs = [basedir + 'z0_0p5000_AH_000_Kinf_m%s_ADV_2_slope_400/' % x for x in [3,4,5]]# + [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [0,1]]
leglabel = [x.replace(basedir + 'z0_0p5000_AH_000_','').replace('_slope_400/','').replace('_',' ').replace('Kinf m','$\kappa_\infty = 10^{-').replace(' ADV 2','}m^2s^{-1}$ ').replace(' ADV 1','}m^2s^{-1}$ BBL V only').replace('ADV 0','}m^2s^{-1}$ no ADV') for x in runs]
# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_%03d/' % x for x in [100,200,400]]
# leglabel = ['1/100','1/200','1/400']
# runs = [basedir + 'z0_0p5000_AH_%03d/' % x for x in [0,10,100]]
# leglabel = ['No Isopycnal Diffusion','$A_H=10$m$^2$s$^{-1}$','$A_H=100$m$^2$s$^{-1}$']
colors = ['k-','r-','b-','g-','y-','m-'];
# runs = [basedir + 'ADV_%01d_AH_000_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]]
# runs = runs + [basedir + 'ADV_%01d_AH_100_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection $A_H=0$m$^2$s$^{-1}$','BBL Advection Only $A_H=0$m$^2$s$^{-1}$','No Advection $A_H=0$m$^2$s$^{-1}$']
# leglabel = leglabel + ['SML and BBL Advection $A_H=100$m$^2$s$^{-1}$','BBL Advection Only $A_H=100$m$^2$s$^{-1}$','No Advection $A_H=100$m$^2$s$^{-1}$']
# colors = ['g-','r-','b-','g--','r--','b--'];

pfr = 4
tifin = 25

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        z = file['scales/z/1.0'][:];
        t = file['scales/sim_time'][:]/lday;
        ym0 = file['tasks']['ym0']

        xlims = [0.,t[tifin]]
        for i in range(0,tifin,pfr):#len(t),pfr):
            if i == 0:
                ab0.plot(ym0[i,0,:],z,colors[ii],label=leglabel[ii])
            else:
                ab0.plot(ym0[i,0,:],z,colors[ii])

        ab0.legend()
        zm1 = file['tasks']['zm1i'][:,0,0];
        zm2 = file['tasks']['zm2i'][:,0,0]
        trT = file['tasks']['trT'][:,0,0]
        Ktr = file['tasks']['Ktr'][:,0,0]
        Katrz = file['tasks']['Katrz'][:,0,0]
        atrzT = file['tasks']['atrzT'][:,0,0]
        mu = zm1/trT
        sigma2 = zm2/trT-(zm1/trT)**2.
        ab1.plot(t,mu,colors[ii])
        ab2.plot(t,np.sqrt(sigma2),colors[ii])

        ab3.plot((t[1:]+t[:-1])/2,0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/1e-4,colors[ii])
        ab3.plot(t,Ktr/trT/1e-4,colors[ii].replace('-','--'))
        ab3.set_xlim(xlims)
        
        # # Analytic solution for constant kappa:
        # sigA = np.sqrt(sz**2 + 2*K0*t*lday)
        # muA = 2*sigA/np.sqrt(2*np.pi)*np.exp(-cz**2/2/sigA**2.) + cz*erf(cz/np.sqrt(2*sigA**2))
        # m2A = cz**2+sigA**2
        # sigAeff = m2A - muA**2
        # KeffA = K0*(1.-2./np.pi*np.exp(-cz**2/sigA**2) - 2.*cz/np.sqrt(2.*np.pi)/sigA*erf(cz/np.sqrt(2*sigA**2))*np.exp(-cz**2/2/sigA**2))
        
        # ab1.plot(t,muA,'--c')
        # ab2.plot(t,np.sqrt(sigAeff),'--c')
        # ab3.plot(t,KeffA/1.e-4,'--c')

# Kinf cases:
ab0.set_xlim([0, 30000.])
ab0.set_ylim([0., 600.])
ab1.set_xlim(xlims)
ab1.set_ylim([200., 350.])
ab2.set_xlim(xlims)
ab2.set_ylim([0, 250.])

xlims = ab0.get_xlim();ylims = ab0.get_ylim();
ab0.text(xlims[0]+0.1*(xlims[1]-xlims[0]),ylims[0]+0.95*(ylims[1]-ylims[0]),panlabs[0],fontsize=15,fontweight='bold')
xlims = ab1.get_xlim();ylims = ab1.get_ylim();
ab1.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.86*(ylims[1]-ylims[0]),panlabs[1],fontsize=15,fontweight='bold')
xlims = ab2.get_xlim();ylims = ab2.get_ylim();
ab2.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.86*(ylims[1]-ylims[0]),panlabs[2],fontsize=15,fontweight='bold')
xlims = ab3.get_xlim();ylims = ab3.get_ylim();
ab3.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.1*(ylims[1]-ylims[0]),panlabs[3],fontsize=15,fontweight='bold')

plt.tight_layout()
plt.savefig(savedir + 'zSpaceAnalysis_KinfADV.pdf');


# Analytic Solution Constant Mixing with Boundary

In [ ]:
# Constant Mixing Time Series:
t = np.linspace(0.,800,500)
K0 = 1.e-3
sz = 50.
cz = 250.

# Analytic solution for constant kappa:
sigA = np.sqrt(sz**2 + 2*K0*t*lday)
muA = 2*sigA/np.sqrt(2*np.pi)*np.exp(-cz**2/2/sigA**2) + cz*erf(cz/np.sqrt(2*sigA**2))
m2A = cz**2+sigA**2
sigAeff = m2A - muA**2
KeffA = K0*(1.-2./np.pi*np.exp(-cz**2/sigA**2) - 2.*cz/np.sqrt(2.*np.pi)/sigA*erf(cz/np.sqrt(2*sigA**2))*np.exp(-cz**2/2/sigA**2))

# # Time series:
# f = plt.figure(figsize=(10,5))
# f.set_facecolor('white')
# plt.plot(t,muA)

# Animation of two gaussians:
z = np.linspace(0.,2000.,500)
for t in np.linspace(0,1000,200):
    sigA = np.sqrt(sz**2 + 2*K0*t*lday)
    sol1 = sz/sigA*np.exp(-(z-cz)**2/2./sigA**2)
    sol2 = sz/sigA*np.exp(-(z+cz)**2/2./sigA**2)

    f = plt.figure(figsize=(5,5))
    f.set_facecolor('white')

    plt.plot(sol1,z,'-r',label='Upper Gaussian')
    plt.plot(sol2,z,'-b',label='Lower Gaussian')
    plt.plot(sol1+sol2,z,'-k',label='Total')
    plt.xlabel('Amplitude')
    plt.ylabel('Height (m)')
    plt.grid()
    plt.title('Time = %03d days' % t)
    plt.xlim(-0.001,0.5)
    plt.ylim(0.,1000.)
    plt.legend()
    plt.savefig('frame%03d.png' % t)

### Y Distribution and moments

In [ ]:
# Plot Y distribution and moments:
fig = plt.figure(figsize=(10, 5), facecolor='w')
ab0 = plt.subplot2grid((3,2),(0,0), colspan=1, rowspan=3)
ab1 = plt.subplot2grid((3,2),(0,1));
ab2 = plt.subplot2grid((3,2),(1,1));
ab3 = plt.subplot2grid((3,2),(2,1));
ab0.set_xlabel('y (km)');ab0.set_ylabel('Tracer Concentration (tr / m)');
ab0.set_title('Tracer Distribution');ab0.set_xlim([0.,Ly/1.e3]);
ab1.set_ylabel('y (km)');ab1.set_title('Centre of mass $\mu_y$');
ab1.grid()
ab2.set_ylabel('y (km)');ab2.set_title('Standard deviation $\sigma_y$');
ab2.grid()
ab3.set_ylabel(r'$\kappa^y_{eff} (m^2s^{-1})$');ab3.set_title('Effective diffusivity from second moment');
ab3.set_xlabel('Days')
ab3.grid()
panlabs = ['a','b','c','d','e','f','g','h','i','j','k','l']

# runs = [basedir + 'z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400/' for x in ['1p0000','0p5000','0p1250']]
# leglabel = ['$z_0=d=500$m','$z_0=d/2=250$m','$z_0=d/8=62.5$m']
# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m%s_ADV_2_slope_400/' % x for x in [3,4,5]] + [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [0,1]]
# leglabel = [x.replace(basedir + 'z0_0p5000_AH_000_','').replace('_slope_400/','').replace('_',' ').replace('Kinf m','$\kappa_\infty = 10^{-').replace(' ADV 2','}m^2s^{-1}$ ').replace(' ADV 1','}m^2s^{-1}$ BBL V only').replace('ADV 0','}m^2s^{-1}$ no ADV') for x in runs]
# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_%03d/' % x for x in [100,200,400]]
# leglabel = ['1/100','1/200','1/400']
# runs = [basedir + 'z0_0p5000_AH_%03d/' % x for x in [0,10,100]]
# leglabel = ['No Isopycnal Diffusion','$A_H=10$m$^2$s$^{-1}$','$A_H=100$m$^2$s$^{-1}$']
# runs = [basedir + 'z0_0p5000_AH_%03d_Kinf_m5_ADV_2_slope_400/' % x for x in [0,1,5,10,50,100]]
# leglabel = ['%sm$^2$s$^{-1}$' % x for x in [0,1,5,10,50,100]]
# colors = ['k-','r-','b-','g-','y-','m-'];
# runs = [basedir + 'ADV_%s_AH_000_slope_200/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
runs = [basedir + 'ADV_%01d_AH_000_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]]
runs = runs + [basedir + 'ADV_%01d_AH_100_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]]
leglabel = ['SML and BBL Advection $A_H=0$m$^2$s$^{-1}$','BBL Advection Only $A_H=0$m$^2$s$^{-1}$','No Advection $A_H=0$m$^2$s$^{-1}$']
leglabel = leglabel + ['SML and BBL Advection $A_H=100$m$^2$s$^{-1}$','BBL Advection Only $A_H=100$m$^2$s$^{-1}$','No Advection $A_H=100$m$^2$s$^{-1}$']
colors = ['g-','r-','b-','g--','r--','b--'];

pfr = 16
tifin = 199

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        zm0 = file['tasks']['zm0']

        xlims = [0.,t[tifin]]
        for i in range(0,tifin,pfr):#len(t),pfr):
            if i == 0:
                ab0.plot(y/1.e3,zm0[i,:,0],colors[ii],label=leglabel[ii])
            else:
                ab0.plot(y/1.e3,zm0[i,:,0],colors[ii])
#        ab0.legend(title='Horizontal Diffusivity $A_H$')
#        ab0.legend(title='Initial Tracer Height $z_0$')
        ab0.legend()
    
        ym1 = file['tasks']['ym1i'][:,0,0];
        ym2 = file['tasks']['ym2i'][:,0,0]
        trT = file['tasks']['trT'][:,0,0]
        mu = ym1/trT
        sigma2 = ym2/trT-(ym1/trT)**2.
        ab1.plot(t,mu/1.e3,colors[ii])
        ab2.plot(t,np.sqrt(sigma2)/1.e3,colors[ii])
        ab3.plot((t[1:]+t[:-1])/2,0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday,colors[ii])
        # print(np.max(0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday))


ab3.set_xlim(xlims)
ab1.set_xlim(xlims)
ab2.set_xlim(xlims)

#ab0.set_xlim([300.,1100.])
#ab0.set_ylim([0.,50.])
xlims = ab0.get_xlim();ylims = ab0.get_ylim();
ab0.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.95*(ylims[1]-ylims[0]),panlabs[0],fontsize=15,fontweight='bold')
xlims = ab1.get_xlim();ylims = ab1.get_ylim();
ab1.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.83*(ylims[1]-ylims[0]),panlabs[1],fontsize=15,fontweight='bold')
xlims = ab2.get_xlim();ylims = ab2.get_ylim();
ab2.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.83*(ylims[1]-ylims[0]),panlabs[2],fontsize=15,fontweight='bold')
xlims = ab3.get_xlim();ylims = ab3.get_ylim();
ab3.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.83*(ylims[1]-ylims[0]),panlabs[3],fontsize=15,fontweight='bold')
plt.tight_layout()
# plt.savefig(savedir + 'ySpaceAnalysis_AH.pdf');


# Diapycnal Fluxes

In [ ]:
fig = plt.figure(figsize=(10, 5), facecolor='w')
ab0 = plt.subplot(1,2,1);
ab1 = plt.subplot(1,2,2);
ab0.set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');ab0.set_xlabel('Tracer Concentration (tr / (ms$^{-2}$))');
ab0.set_title('Tracer Distribution Day 320')
ab0.set_ylim([5.,20.]);
ab1.set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');ab0.set_xlabel('Diapycnal Tracer Flux (tr / (s$^{-2}$))');
ab1.set_title('Diapycnal Tracer Flux Day 320')
ab1.set_ylim([5.,20.]);
ab1.grid()            
panlabs = ['a','b','c','d','e','f','g','h','i','j','k','l']

# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
runs = [basedir + 'z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400/' for x in ['1p0000','0p5000','0p1250']]
leglabel = ['$z_0=d=500$m','$z_0=d/2=250$m','$z_0=d/8=62.5$m']
colors = ['g-','r-','b-','k-','y-','m-'];
# runs = [runs[1]]

pfr = 20
tifin = 21

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    b = np.linspace(0,N2*np.cos(theta)*Lz + N2*np.sin(theta)*Ly,64);db = b[1]-b[0]
    ba = (b[:-1]+b[1:])/2

    with h5py.File(runs[ii] + 'ifields.h5', mode='r') as file:
        B = file['tasks']['B'][0,:,:]
        B = (B[1:,:] + B[:-1,:])/2
        B = (B[:,1:] + B[:,:-1])/2
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
    dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
    dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        dt = (t[1]-t[0])*lday;
        trT = file['tasks']['trT'][:,0,0]

        mub = np.zeros(len(t))
        sigma2b = np.zeros(len(t))
        xlims = [0.,t[tifin]]
        for i in [20]:#range(0,tifin,pfr):#len(t),pfr):
            tr = file['tasks']['tr'][i,:,:]
            tr = (tr[1:,:] + tr[:-1,:])/2
            tr = (tr[:,1:] + tr[:,:-1])/2
            tr = tr*dA
            trCm = np.array([np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]])
            tr = file['tasks']['tr'][i+1,:,:]
            tr = (tr[1:,:] + tr[:-1,:])/2
            tr = (tr[:,1:] + tr[:,:-1])/2
            tr = tr*dA
            trCp = np.array([np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]])

            # bpl = np.linspace(b.min()/1.e-4,b.max()/1.e-4,300)
            # trC_smooth = spline((b[:-1]+b[1:])/2/1.e-4,(trCp+trCm)/2.,bpl);
            if i == 20:
                ab0.plot((trCp+trCm)/2.,(b[:-1]+b[1:])/2/1.e-4,colors[ii],label=leglabel[ii])
                # ab0.plot(trC_smooth,bpl,colors[ii],label=leglabel[ii])
            else:
                ab0.plot((trCp+trCm)/2.,(b[:-1]+b[1:])/2/1.e-4,colors[ii])
                # ab0.plot(trC_smooth,bpl,colors[ii])
            dPdt = (trCp-trCm)/dt
            trF = -np.cumsum(dPdt*db)
            ab1.plot(trF,(b[:-1]+b[1:])/2/1.e-4,colors[ii])
#            ab1.plot(dPdt,(b[:-1]+b[1:])/2/1.e-4,colors[ii])
        ab0.legend()
        ab0.set_xlim([0.,1.0e10])
        ab1.set_xlim([-0.02,0.04])

xlims = ab0.get_xlim();ylims = ab0.get_ylim();
ab0.text(xlims[0]+0.07*(xlims[1]-xlims[0]),ylims[0]+0.95*(ylims[1]-ylims[0]),panlabs[0],fontsize=15,fontweight='bold')
xlims = ab1.get_xlim();ylims = ab1.get_ylim();
ab1.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.83*(ylims[1]-ylims[0]),panlabs[1],fontsize=15,fontweight='bold')

plt.tight_layout()
plt.savefig(savedir + 'bSpaceFluxes_z0.pdf');


In [ ]:
# Analytic solution plotting:

In [ ]:
mu0z/np.sqrt(2.*sig0z**2.)

In [ ]:
# parameters:
d = 500.;kappa = 1.e-3;AH = 100.
mu0z = d/2;mu0y = 0.
sig0z = 10.;sig0y = 100.
slope = 1/400.;

# Derived parameters:
theta = np.arctan(slope);
K11 = kappa + AH*np.cos(theta)**2.
K12 = - AH*np.sin(theta)*np.cos(theta)
K22 = kappa + AH*np.sin(theta)**2.

# Time axis:
t = np.linspace(0.,1000.,100.)*1.e5

# Moment solution time series:
sig2 = sig0z**2 + 2.*K22*t;
sig = np.sqrt(sig2);
trT = 2.*np.pi*sig0z*sig0y;
# muz = 2*sig/np.sqrt(2.*pi)*np.exp(-mu0z**2./2./sig2)+mu0z*erf(mu0z/np.sqrt(2.*sig2)); #INCORRECT NORMALIZATION
# sigz2 = mu0z**2.+sig2-muz**2. #INCORRECT NORMALIZATION

muy = (mu0y + trT*K12/K22*(2*sig*(np.exp(-mu0z**2/2/sig2)-np.exp(-mu0z**2./2./sig0z**2.))+np.sqrt(2*np.pi)*mu0z*(erf(mu0z/np.sqrt(2*sig2))-erf(mu0z/np.sqrt(2.*sig0z**2.)))))/trT

plt.plot(t,muy)




### Analytic plot of B derivatives to check BBL approximations for isopycnal mixing

In [ ]:
theta = np.arctan(1./100.);N2 = 1.e-6;K0 = 1.e-3;Pr0=1.
q0 = (N2*np.sin(theta)*np.sin(theta)/4.0/Pr0/K0/K0)**(1.0/4.0)

z = np.linspace(0.,2./q0,200.)
f  = np.exp(-q0*z)*(np.cos(q0*z)+np.sin(q0*z))
fz = -2.*q0*np.exp(-q0*z)*np.sin(q0*z)
V = -K0*fz/np.tan(theta);
V = V/np.max(V)

Bz = 1-f
Bz2d = (1-f)**2./(np.tan(theta)**2.+(1-f)**2.)
ByBzd = (1-f)/(np.tan(theta)+(1./np.tan(theta))*(1-f)**2.)
By2d = 1/(1+(1./np.tan(theta))**2.*(1-f)**2.)
ByBzdd = fz*((1./np.tan(theta))*(1-f)**2.-np.tan(theta))/(np.tan(theta)+(1./np.tan(theta))*(1-f)**2.)**2.
By2dd = 2*(1-f)*fz/(1+(1./np.tan(theta))**2.*(1-f)**2.)**2.

fig = plt.figure(figsize=(5,7))
fig.set_facecolor('white')
plt.plot(Bz,z,'-k',linewidth=3,label=r'$B_z/N^2\cos{\theta}$')
plt.plot(V,z,'--k',linewidth=3,label=r'$V/max(V)$')
plt.plot(Bz2d,z,'-r',linewidth=2,label=r'$B_z^2/|\nabla B|^2$')
plt.plot(ByBzd,z,'-b',linewidth=2,label=r'$B_z B_y/|\nabla B|^2$')
plt.plot(By2d,z,'-m',linewidth=2,label=r'$B_y^2/|\nabla B|^2$')
plt.plot(ByBzdd,z,'-c',linewidth=2,label=r'$\frac{\partial}{\partial z}(B_z B_y/|\nabla B|^2)$')
plt.plot(By2dd,z,'-g',linewidth=2,label=r'$\frac{\partial}{\partial z}(B_y^2/|\nabla B|^2)$')
plt.plot([0.,1.],[1./q0,1./q0],'-k')
plt.legend()
ax = plt.gca()
ax.set_ylabel('Height $z$ (m)')
ax.set_ylim([0.,np.max(z)])
plt.text(0.03,1./q0+3,r'$\tan{\theta} = 1/%3.0f$' % (1./np.tan(theta)),fontsize='x-large')
plt.text(0.03,1./q0+1,r'$q_0^{-1} = %2.1f$m' % (1./q0),fontsize='x-large')
plt.savefig('IsopycnalMixing_Bgradients_Sl100.png')


# Ununsed/Old plotting code

In [ ]:
# Testing new buoyancy binning code:
import imp
imp.reload(bbin)
run = basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400/'

params = np.load(run + 'runparams.npz');locals().update(params)

with h5py.File(run + 'ifields.h5', mode='r') as file:
    B = file['tasks']['B'][0,:,:]
    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

with h5py.File(run + 'snapshots.h5', mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]
    
    i = 1
    tr = file['tasks']['tr'][i,:,:]

b, trp = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0)

f = plt.figure(figsize=(10,5),facecolor='white')
ax0 = plt.subplot2grid((1,3),(0,0))
ax0.plot(trp,b,'.-k')
ax0.set_ylim([0., N2*np.cos(theta)*Lz*2])

ym, zm = np.meshgrid(y,z)
zt = np.cos(theta)*zm + np.sin(theta)*ym;
yt = -np.sin(theta)*zm + np.cos(theta)*ym # Rotated Coordinates
B = N2*np.sin(theta)*ym + N2*np.cos(theta)*(zm + np.exp(-q0*zm)*np.cos(q0*zm)/q0)

axisTR = plt.subplot2grid((1,3),(0,1), colspan=2, rowspan=1)
p = axisTR.contourf(yt/1.e3, zt, tr.T, cmap='magma_r', levels=np.arange(0.,np.max(tr),np.max(tr)/26),extend='both')#np.max(tr)/);#GnBu
axisTR.set_ylim([0., Lz*2])

# Add buoyancy contours
plt.contour(yt/1.0e3, zt, B, 15, colors='k')
axisTR.plot(y/1.0e3, slope*y,'k-', linewidth=4.)
plt.colorbar(p, ax = axisTR)
#plt.title(run + 'Day %d' % t[i])